In [4]:
import os
import pickle
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import pearsonr
print(pd.__version__)

1.3.5


In [5]:
def age_debiasing(predicted_ages, chronological_ages):
    
    predicted_ages = predicted_ages.to_numpy()
    chronological_ages = chronological_ages.to_numpy()
    
    ones = np.ones(len(chronological_ages))
    chronological_ages_squared = np.square(chronological_ages)
    chronological_ages_confounds = np.array([ones, chronological_ages])
    
    ones_predicted = np.ones(len(predicted_ages))
    predicted_ages_squared = np.square(predicted_ages)
    predicted_ages_confounds = np.array([ones_predicted, predicted_ages])
    
    predicted_ages = np.reshape(predicted_ages, (len(predicted_ages), 1) )
    beta = np.dot( np.linalg.pinv(chronological_ages_confounds.T), predicted_ages )

    corrected_predicted_ages = (predicted_ages.flatten() - beta[0])/beta[1]
    
    return corrected_predicted_ages

In [6]:
T1_nonlinear="1"
T1_linear="2"
jacobian="56"
vbm="4"

T2_nonlinear="5"
T2_lesions="57"
swi="6"

rsfmri_0="7"
rsfmri_1="8"
rsfmri_2="9"
rsfmri_3="10"
rsfmri_4="11"
rsfmri_5="12"
rsfmri_6="13"
rsfmri_7="14"
rsfmri_8="15"
rsfmri_9="16"
rsfmri_10="17"
rsfmri_11="18"
rsfmri_12="19"
rsfmri_13="20"
rsfmri_14="21"
rsfmri_15="22"
rsfmri_16="23"
rsfmri_17="24"
rsfmri_18="25"
rsfmri_19="26"
rsfmri_20="27"
rsfmri_21="28"
rsfmri_22="29"
rsfmri_23="30"
rsfmri_24="31"

tfmri_1="32"
tfmri_2="33"
tfmri_5="34"

tfmri_c_1="35"
tfmri_c_2="36"
tfmri_c_5="37"

tracts="3"

tbss_FA_s="38"
tbss_ICVF_s="39"
tbss_ISOVF_s="40"
tbss_L1_s="41"
tbss_L2_s="42"
tbss_L3_s="43"
tbss_MD_s="44"
tbss_MO_s="45"
tbss_OD_s="46"

tbss_FA="47"
tbss_ICVF="48"
tbss_ISOVF="49"
tbss_L1="50"
tbss_L2="51"
tbss_L3="52"
tbss_MD="53"
tbss_MO="54"
tbss_OD="55"

dfs = [

T1_nonlinear,
T1_linear,
jacobian,
vbm,

T2_nonlinear,
T2_lesions,
swi,

rsfmri_0,
rsfmri_1,
rsfmri_2,
rsfmri_3,
rsfmri_4,
rsfmri_5,
rsfmri_6,
rsfmri_7,
rsfmri_8,
rsfmri_9,
rsfmri_10,
rsfmri_11,
rsfmri_12,
rsfmri_13,
rsfmri_14,
rsfmri_15,
rsfmri_16,
rsfmri_17,
rsfmri_18,
rsfmri_19,
rsfmri_20,
rsfmri_21,
rsfmri_22,
rsfmri_23,
rsfmri_24,

tfmri_1,
tfmri_2,
tfmri_5,

tfmri_c_1,
tfmri_c_2,
tfmri_c_5,

tracts,

tbss_FA_s,
tbss_ICVF_s,
tbss_ISOVF_s,
tbss_L1_s,
tbss_L2_s,
tbss_L3_s,
tbss_MD_s,
tbss_MO_s,
tbss_OD_s,

tbss_FA,
tbss_ICVF,
tbss_ISOVF,
tbss_L1,
tbss_L2,
tbss_L3,
tbss_MD,
tbss_MO,
tbss_OD
]

names = [

'T1_nonlinear',
'T1_linear',
'jacobian',
'vbm',

'T2_nonlinear',
'T2_lesions',
'swi',

'rsfmri_0',
'rsfmri_1',
'rsfmri_2',
'rsfmri_3',
'rsfmri_4',
'rsfmri_5',
'rsfmri_6',
'rsfmri_7',
'rsfmri_8',
'rsfmri_9',
'rsfmri_10',
'rsfmri_11',
'rsfmri_12',
'rsfmri_13',
'rsfmri_14',
'rsfmri_15',
'rsfmri_16',
'rsfmri_17',
'rsfmri_18',
'rsfmri_19',
'rsfmri_20',
'rsfmri_21',
'rsfmri_22',
'rsfmri_23',
'rsfmri_24',

'tfmri_1',
'tfmri_2',
'tfmri_5',

'tfmri_c_1',
'tfmri_c_2',
'tfmri_c_5',

'tracts',

'tbss_FA_s',
'tbss_ICVF_s',
'tbss_ISOVF_s',
'tbss_L1_s',
'tbss_L2_s',
'tbss_L3_s',
'tbss_MD_s',
'tbss_MO_s',
'tbss_OD_s',

'tbss_FA',
'tbss_ICVF',
'tbss_ISOVF',
'tbss_L1',
'tbss_L2',
'tbss_L3',
'tbss_MD',
'tbss_MO',
'tbss_OD'
]

dfs1, dfs2, dfs3 = [], [], []

dfs1 = dfs.copy()
dfs2 = dfs.copy()
dfs3 = dfs.copy()

for idx in range(len(dfs)):
    
    dfs1[idx] = '../predictions/AM2-' + dfs1[idx] + '_predictions/output_statistics_validation.csv'
    dfs2[idx] = '../predictions/AM3-' + dfs2[idx] + '_predictions/output_statistics_validation.csv'
    dfs3[idx] = '../predictions/AM4-' + dfs3[idx] + '_predictions/output_statistics_validation.csv'
    
for idx in range(len(dfs)):
    dfs1[idx] = pd.read_csv(dfs1[idx])
    dfs2[idx] = pd.read_csv(dfs2[idx])
    dfs3[idx] = pd.read_csv(dfs3[idx])

In [7]:
for idx in range(len(dfs)):
    dfs[idx] = pd.merge(dfs1[idx], dfs2[idx], on='Unnamed: 0', how='inner')
    dfs[idx] = pd.merge(dfs[idx], dfs3[idx], on='Unnamed: 0', how='inner')
    
    dfs[idx].rename({
        'target_age_x': 'target_age_original',
        'output_age_x': 'output_age_1',
        'age_delta_x': 'age_delta_1',
        'loss_x': 'loss_1',
        'target_age_y': 'target_age_2',
        'output_age_y': 'output_age_2',
        'age_delta_y': 'age_delta_2',
        'loss_y': 'loss_2',
        'target_age': 'target_age_3',
        'output_age': 'output_age_3',
        'age_delta': 'age_delta_3',
        'loss': 'loss_3',
    }, axis=1, inplace=True)
    
    dfs[idx].drop(['target_age_2', 'target_age_3'], axis=1, inplace=True)
    
    dfs[idx].rename({
        'target_age_original': 'target_age',
    }, axis=1, inplace=True)
    
    dfs[idx]['mean_output_age'] = dfs[idx][['output_age_1', 'output_age_2', 'output_age_3']].mean(axis=1)
    dfs[idx]['mean_age_delta'] = dfs[idx]['mean_output_age'] - dfs[idx]['target_age']

In [8]:
for idx in range(len(dfs)):    
    dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].mean_output_age, dfs[idx].target_age) 
    dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

In [9]:
print("{:13} : {:15}  |  {:13}  |  {:13}  |  {:13}".format('Modality', 'Run1', "Run2", "Run3", "MEAN"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} : {:7.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  MEAN --- {:.3f} ± {:.3f}".format(
        names[idx],
        np.abs(dfs[idx].age_delta_1).mean(), np.abs(dfs[idx].age_delta_1).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_2).mean(), np.abs(dfs[idx].age_delta_2).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_3).mean(), np.abs(dfs[idx].age_delta_3).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : Run1             |  Run2           |  Run3           |  MEAN         
-----------------------------------------------------------------------------------------------
T1_nonlinear  :   2.389 ± 0.052  |  2.416 ± 0.052  |  2.467 ± 0.052  |  MEAN --- 2.208 ± 0.047
T1_linear     :   2.407 ± 0.052  |  2.523 ± 0.054  |  2.598 ± 0.057  |  MEAN --- 2.270 ± 0.049
jacobian      :   3.436 ± 0.074  |  3.546 ± 0.076  |  3.521 ± 0.073  |  MEAN --- 3.334 ± 0.070
vbm           :   2.698 ± 0.059  |  2.714 ± 0.059  |  2.754 ± 0.060  |  MEAN --- 2.524 ± 0.055
T2_nonlinear  :   2.317 ± 0.051  |  2.299 ± 0.049  |  2.353 ± 0.050  |  MEAN --- 2.151 ± 0.046
T2_lesions    :   4.146 ± 0.085  |  4.208 ± 0.088  |  4.106 ± 0.087  |  MEAN --- 4.066 ± 0.084
swi           :   3.303 ± 0.068  |  3.359 ± 0.071  |  3.238 ± 0.071  |  MEAN --- 3.038 ± 0.064
rsfmri_0      :   4.408 ± 0.090  |  4.435 ± 0.093  |  4.501 ± 0.090  |  MEAN --- 4.242 ± 0.085
rsfmri_1      :   4.609 ± 0.095  |  4.496 ± 0.093  |  4.59

In [10]:
print("{:13} : {:14}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : MEAN (MAE)      |  MEAN DECONF    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.208 ± 0.047  |   2.365 ± 0.051 
T1_linear     :  2.270 ± 0.049  |   2.481 ± 0.052 
jacobian      :  3.334 ± 0.070  |   4.031 ± 0.085 
vbm           :  2.524 ± 0.055  |   2.805 ± 0.061 
T2_nonlinear  :  2.151 ± 0.046  |   2.292 ± 0.048 
T2_lesions    :  4.066 ± 0.084  |   5.513 ± 0.116 
swi           :  3.038 ± 0.064  |   3.510 ± 0.075 
rsfmri_0      :  4.242 ± 0.085  |   5.845 ± 0.122 
rsfmri_1      :  4.306 ± 0.088  |   6.054 ± 0.132 
rsfmri_2      :  4.345 ± 0.088  |   6.141 ± 0.132 
rsfmri_3      :  4.425 ± 0.090  |   6.364 ± 0.138 
rsfmri_4      :  4.303 ± 0.088  |   6.121 ± 0.125 
rsfmri_5      :  4.374 ± 0.090  |   6.328 ± 0.133 
rsfmri_6      :  4.422 ± 0.088  |   6.306 ± 0.136 
rsfmri_7      :  4.251 ± 0.086  |   5.977 ± 0.123 
rsfmri_8      :  4.402 ± 0.088  |   6.204 ± 0.133 
rsfmri_9      :  4.413 ± 0.090  |   6

In [11]:
mae_weighting = 37.137671232876755 # for female subjects
# mae_weighting = 36.28150684931529 # for male subjects


print("{:13} : {:14}  |  {:15}  |  {:7}  |  {:13} ".format('Modality', 'MAE', "RMSE", 
                                                            "CORR (R)", "MAE weighted"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:5.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0],
        np.abs(dfs[idx].mean_age_delta).mean() / mae_weighting,
    )
         )

Modality      : MAE             |  RMSE             |  CORR (R)  |  MAE weighted  
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.208 ± 0.047  |   2.757 ± 0.548   |   0.928   |   0.059 
T1_linear     :  2.270 ± 0.049  |   2.863 ± 0.582   |   0.923   |   0.061 
jacobian      :  3.334 ± 0.070  |   4.158 ± 0.825   |   0.828   |   0.090 
vbm           :  2.524 ± 0.055  |   3.191 ± 0.650   |   0.902   |   0.068 
T2_nonlinear  :  2.151 ± 0.046  |   2.692 ± 0.539   |   0.933   |   0.058 
T2_lesions    :  4.066 ± 0.084  |   5.043 ± 0.994   |   0.733   |   0.109 
swi           :  3.038 ± 0.064  |   3.790 ± 0.738   |   0.859   |   0.082 
rsfmri_0      :  4.242 ± 0.085  |   5.191 ± 0.988   |   0.714   |   0.114 
rsfmri_1      :  4.306 ± 0.088  |   5.323 ± 1.030   |   0.695   |   0.116 
rsfmri_2      :  4.345 ± 0.088  |   5.348 ± 1.033   |   0.693   |   0.117 
rsfmri_3      :  4.425 ± 0.090  |   5.448 ± 1.051   |   0.678   |   0.1

In [12]:
maes=[]
prs = []
for idx in range(len(dfs)):
    mae = np.abs(dfs[idx].mean_age_delta).mean()
    maes.append(mae)
    pr = pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0]
    prs.append(pr)
    
print(pearsonr(maes, prs)[0])

-0.9953304343209817


In [13]:
print("{:13} : {:14}  |  {:15}  |  {:13} ".format('Modality', 'MAE (DECON)', "RMSE (DECON)", "PR (DECON)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  RMSE (DECON)     |  PR (DECON)    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.365 ± 0.051  |   2.968 ± 0.591   |   0.928 
T1_linear     :  2.481 ± 0.052  |   3.098 ± 0.616   |   0.923 
jacobian      :  4.031 ± 0.085  |   5.021 ± 1.009   |   0.828 
vbm           :  2.805 ± 0.061  |   3.535 ± 0.705   |   0.902 
T2_nonlinear  :  2.292 ± 0.048  |   2.859 ± 0.565   |   0.933 
T2_lesions    :  5.513 ± 0.116  |   6.877 ± 1.363   |   0.733 
swi           :  3.510 ± 0.075  |   4.411 ± 0.875   |   0.859 
rsfmri_0      :  5.845 ± 0.122  |   7.260 ± 1.426   |   0.714 
rsfmri_1      :  6.054 ± 0.132  |   7.654 ± 1.541   |   0.695 
rsfmri_2      :  6.141 ± 0.132  |   7.710 ± 1.547   |   0.693 
rsfmri_3      :  6.364 ± 0.138  |   8.025 ± 1.623   |   0.678 
rsfmri_4      :  6.121 ± 0.125  |   7.560 ± 1.489   |   0.700 
rsfmri_5      :  6.328 ± 0.133  |   7.884 ± 1.569   |   0.685 
rsfmri_6      :

# Saving Dataframe for Later Use

In [14]:
d = {'modality':names, 'dataframe':dfs}
df = pd.DataFrame(data=d)

In [15]:
df['dataframe'].iloc[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23138231,64.763813,64.746210,-0.017605,0.000310,62.432076,-2.331737,5.436999,62.578835,-2.184978,4.774130,63.252374,-1.511439,63.135638,-1.628175
1,22945960,66.762215,68.205010,1.442795,2.081657,68.484856,1.722641,2.967492,68.001320,1.239105,1.535382,68.230395,1.468181,68.898975,2.136760
2,22018561,61.399429,63.318413,1.918984,3.682498,61.221520,-0.177910,0.031652,58.990425,-2.409004,5.803301,61.176786,-0.222643,60.732612,-0.666817
3,21597696,59.327186,58.370632,-0.956554,0.914995,58.096596,-1.230590,1.514352,59.575203,0.248017,0.061513,58.680810,-0.646375,57.842880,-1.484306
4,23132897,66.550571,67.496980,0.946408,0.895688,72.144680,5.594105,31.294016,68.320290,1.769719,3.131905,69.320650,2.770079,70.161225,3.610654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,22600330,67.113584,69.930145,2.816561,7.933015,70.230310,3.116724,9.713969,68.798744,1.685160,2.839763,69.653066,2.539482,70.546082,3.432497
1251,25605778,57.032104,55.545610,-1.486495,2.209668,54.718300,-2.313804,5.353689,57.129770,0.097665,0.009538,55.797893,-1.234210,54.505164,-2.526940
1252,21256215,70.362900,75.201740,4.838837,23.414343,73.045555,2.682656,7.196641,72.413734,2.050835,4.205924,73.553676,3.190777,75.062039,4.699139
1253,24828516,63.511301,62.478394,-1.032908,1.066899,59.926506,-3.584795,12.850758,58.432228,-5.079073,25.796985,60.279043,-3.232259,59.693244,-3.818057


In [16]:
df

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [17]:
type(df.dataframe.iloc[0])

pandas.core.frame.DataFrame

In [18]:
df.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23138231,64.763813,64.746210,-0.017605,0.000310,62.432076,-2.331737,5.436999,62.578835,-2.184978,4.774130,63.252374,-1.511439,63.135638,-1.628175
1,22945960,66.762215,68.205010,1.442795,2.081657,68.484856,1.722641,2.967492,68.001320,1.239105,1.535382,68.230395,1.468181,68.898975,2.136760
2,22018561,61.399429,63.318413,1.918984,3.682498,61.221520,-0.177910,0.031652,58.990425,-2.409004,5.803301,61.176786,-0.222643,60.732612,-0.666817
3,21597696,59.327186,58.370632,-0.956554,0.914995,58.096596,-1.230590,1.514352,59.575203,0.248017,0.061513,58.680810,-0.646375,57.842880,-1.484306
4,23132897,66.550571,67.496980,0.946408,0.895688,72.144680,5.594105,31.294016,68.320290,1.769719,3.131905,69.320650,2.770079,70.161225,3.610654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,22600330,67.113584,69.930145,2.816561,7.933015,70.230310,3.116724,9.713969,68.798744,1.685160,2.839763,69.653066,2.539482,70.546082,3.432497
1251,25605778,57.032104,55.545610,-1.486495,2.209668,54.718300,-2.313804,5.353689,57.129770,0.097665,0.009538,55.797893,-1.234210,54.505164,-2.526940
1252,21256215,70.362900,75.201740,4.838837,23.414343,73.045555,2.682656,7.196641,72.413734,2.050835,4.205924,73.553676,3.190777,75.062039,4.699139
1253,24828516,63.511301,62.478394,-1.032908,1.066899,59.926506,-3.584795,12.850758,58.432228,-5.079073,25.796985,60.279043,-3.232259,59.693244,-3.818057


In [19]:
df.iloc[0].modality

'T1_nonlinear'

In [20]:
type(df.iloc[0].dataframe)

pandas.core.frame.DataFrame

In [21]:
df

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [22]:
df.to_pickle('female_validation.pkl', protocol = 4)

In [23]:
df2 = pd.read_pickle('female_validation.pkl')

In [24]:
df2

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [25]:
df2.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23138231,64.763813,64.746210,-0.017605,0.000310,62.432076,-2.331737,5.436999,62.578835,-2.184978,4.774130,63.252374,-1.511439,63.135638,-1.628175
1,22945960,66.762215,68.205010,1.442795,2.081657,68.484856,1.722641,2.967492,68.001320,1.239105,1.535382,68.230395,1.468181,68.898975,2.136760
2,22018561,61.399429,63.318413,1.918984,3.682498,61.221520,-0.177910,0.031652,58.990425,-2.409004,5.803301,61.176786,-0.222643,60.732612,-0.666817
3,21597696,59.327186,58.370632,-0.956554,0.914995,58.096596,-1.230590,1.514352,59.575203,0.248017,0.061513,58.680810,-0.646375,57.842880,-1.484306
4,23132897,66.550571,67.496980,0.946408,0.895688,72.144680,5.594105,31.294016,68.320290,1.769719,3.131905,69.320650,2.770079,70.161225,3.610654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,22600330,67.113584,69.930145,2.816561,7.933015,70.230310,3.116724,9.713969,68.798744,1.685160,2.839763,69.653066,2.539482,70.546082,3.432497
1251,25605778,57.032104,55.545610,-1.486495,2.209668,54.718300,-2.313804,5.353689,57.129770,0.097665,0.009538,55.797893,-1.234210,54.505164,-2.526940
1252,21256215,70.362900,75.201740,4.838837,23.414343,73.045555,2.682656,7.196641,72.413734,2.050835,4.205924,73.553676,3.190777,75.062039,4.699139
1253,24828516,63.511301,62.478394,-1.032908,1.066899,59.926506,-3.584795,12.850758,58.432228,-5.079073,25.796985,60.279043,-3.232259,59.693244,-3.818057


In [24]:
df2.iloc[0].modality

'T1_nonlinear'

In [25]:
type(df2.iloc[0].dataframe)

pandas.core.frame.DataFrame

# Test what happens if debiasing before deconfounding

In [26]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23138231,64.763813,64.746210,-0.017605,0.000310,62.432076,-2.331737,5.436999,62.578835,-2.184978,4.774130,63.252374,-1.511439,63.135638,-1.628175
1,22945960,66.762215,68.205010,1.442795,2.081657,68.484856,1.722641,2.967492,68.001320,1.239105,1.535382,68.230395,1.468181,68.898975,2.136760
2,22018561,61.399429,63.318413,1.918984,3.682498,61.221520,-0.177910,0.031652,58.990425,-2.409004,5.803301,61.176786,-0.222643,60.732612,-0.666817
3,21597696,59.327186,58.370632,-0.956554,0.914995,58.096596,-1.230590,1.514352,59.575203,0.248017,0.061513,58.680810,-0.646375,57.842880,-1.484306
4,23132897,66.550571,67.496980,0.946408,0.895688,72.144680,5.594105,31.294016,68.320290,1.769719,3.131905,69.320650,2.770079,70.161225,3.610654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,22600330,67.113584,69.930145,2.816561,7.933015,70.230310,3.116724,9.713969,68.798744,1.685160,2.839763,69.653066,2.539482,70.546082,3.432497
1251,25605778,57.032104,55.545610,-1.486495,2.209668,54.718300,-2.313804,5.353689,57.129770,0.097665,0.009538,55.797893,-1.234210,54.505164,-2.526940
1252,21256215,70.362900,75.201740,4.838837,23.414343,73.045555,2.682656,7.196641,72.413734,2.050835,4.205924,73.553676,3.190777,75.062039,4.699139
1253,24828516,63.511301,62.478394,-1.032908,1.066899,59.926506,-3.584795,12.850758,58.432228,-5.079073,25.796985,60.279043,-3.232259,59.693244,-3.818057


In [29]:
for idx in range(len(dfs)):
    dfs[idx]['output_age_decon_1'] = age_debiasing(dfs[idx].output_age_1, dfs[idx].target_age)
    dfs[idx]['output_age_decon_2'] = age_debiasing(dfs[idx].output_age_2, dfs[idx].target_age) 
    dfs[idx]['output_age_decon_3'] = age_debiasing(dfs[idx].output_age_3, dfs[idx].target_age) 
    dfs[idx]['mean_output_age_decon'] = dfs[idx][['output_age_decon_1', 'output_age_decon_2', 'output_age_decon_3']].mean(axis=1)
    dfs[idx]['mean_age_delta_decon'] = dfs[idx]['mean_output_age_decon'] - dfs[idx]['target_age']

In [30]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon,output_age_decon_1,output_age_decon_2,output_age_decon_3,mean_output_age_decon,mean_age_delta_decon
0,23138231,64.763813,64.746210,-0.017605,0.000310,62.432076,-2.331737,5.436999,62.578835,-2.184978,4.774130,63.252374,-1.511439,63.135638,-1.628175,64.934619,62.000106,62.419660,63.118128,-1.645684
1,22945960,66.762215,68.205010,1.442795,2.081657,68.484856,1.722641,2.967492,68.001320,1.239105,1.535382,68.230395,1.468181,68.898975,2.136760,68.854158,68.975271,68.867278,68.898902,2.136688
2,22018561,61.399429,63.318413,1.918984,3.682498,61.221520,-0.177910,0.031652,58.990425,-2.409004,5.803301,61.176786,-0.222643,60.732612,-0.666817,63.316628,60.605073,58.152854,60.691518,-0.707911
3,21597696,59.327186,58.370632,-0.956554,0.914995,58.096596,-1.230590,1.514352,59.575203,0.248017,0.061513,58.680810,-0.646375,57.842880,-1.484306,57.709764,57.003940,58.848185,57.853963,-1.473222
4,23132897,66.550571,67.496980,0.946408,0.895688,72.144680,5.594105,31.294016,68.320290,1.769719,3.131905,69.320650,2.770079,70.161225,3.610654,68.051813,73.192817,69.246549,70.163726,3.613156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,22600330,67.113584,69.930145,2.816561,7.933015,70.230310,3.116724,9.713969,68.798744,1.685160,2.839763,69.653066,2.539482,70.546082,3.432497,70.809094,70.986715,69.815456,70.537089,3.423504
1251,25605778,57.032104,55.545610,-1.486495,2.209668,54.718300,-2.313804,5.353689,57.129770,0.097665,0.009538,55.797893,-1.234210,54.505164,-2.526940,54.508428,53.110825,55.940438,54.519897,-2.512207
1252,21256215,70.362900,75.201740,4.838837,23.414343,73.045555,2.682656,7.196641,72.413734,2.050835,4.205924,73.553676,3.190777,75.062039,4.699139,76.782907,74.230977,74.113868,75.042584,4.679684
1253,24828516,63.511301,62.478394,-1.032908,1.066899,59.926506,-3.584795,12.850758,58.432228,-5.079073,25.796985,60.279043,-3.232259,59.693244,-3.818057,62.364712,59.112711,57.489129,59.655517,-3.855784


In [31]:
print("{:13} : {:14}  |  {:14}  |  {:13}  |   {:13}  |  {:}  |  {:} ".format('Modality', 'MAE (DECON)', 'MAE (DECON B)', "MSE (DECON)", "MSE (DECON B)",  "PR (DECON)", "PR (DECON B)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:.3f} ".format(
        names[idx],
        
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta_decon).mean(), np.abs(dfs[idx].mean_age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  MAE (DECON B)   |  MSE (DECON)    |   MSE (DECON B)  |  PR (DECON)  |  PR (DECON B) 
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  2.365 ± 0.051  |   2.366 ± 0.051   |   8.809 ± 0.349  |   8.814 ± 0.349   |   0.928   |   0.928 
T1_linear     :  2.481 ± 0.052  |   2.481 ± 0.052   |   9.599 ± 0.379  |   9.595 ± 0.379   |   0.923   |   0.923 
jacobian      :  4.031 ± 0.085  |   4.033 ± 0.085   |   25.213 ± 1.018  |   25.228 ± 1.019   |   0.828   |   0.828 
vbm           :  2.805 ± 0.061  |   2.805 ± 0.061   |   12.495 ± 0.498  |   12.498 ± 0.498   |   0.902   |   0.902 
T2_nonlinear  :  2.292 ± 0.048  |   2.292 ± 0.048   |   8.174 ± 0.319  |   8.178 ± 0.319   |   0.933   |   0.933 
T2_lesions    :  5.513 ± 0.116  |   5.518 ± 0.116   |   47.293 ± 1.859  |   47.386 ± 1.863   |   0.733   |   0.732 
swi           :  3.510 ± 0.075  |   3.514 ± 0.076   |   19.457 ± 0.765  |   19.515 ± 0.767   |